In [1]:
import pandas as pd
import numpy as np

#pd.set_option("max_colwidth",1000000)
#pd.set_option('max_columns', 50)

### Escolha de colunas e leitura “leve”

In [2]:
import pandas as pd
# pandas ≥ 2.1 recomendado
pd.set_option("display.width", 140)

PATH = "../data/raw/2007Q1.parquet"

# Apenas o que interessa para montar o target
cols = [
    "LOAN_ID",                # id do empréstimo
    "ACT_PERIOD",             # mês de referência – formato MMYYYY
    "DLQ_STATUS",             # dias de atraso / códigos
]

# dtype_backend="pyarrow" → strings continuam como Arrow, economizando RAM
perf = pd.read_parquet(
    PATH,
    columns=cols,
    engine="pyarrow",
    dtype_backend="pyarrow",
)
print(perf.shape)

(16588215, 3)


### Parsing robusto do campo de data

In [3]:
import numpy as np

def parse_act_period(s: pd.Series) -> pd.Series:
    """
    Converte MMYYYY (formato oficial) ou YYYYMM (legado) em datetime64[ns]
    no último dia do mês. Retorna NaT para valores fora de padrão.
    """
    s = s.astype(str).str.zfill(6)
    is_mmYYYY = s.str[:2].astype(int).between(1, 12)  # 01-12 ⇒ MMYYYY
    mmYYYY = s[is_mmYYYY]
    yyyyMM = s[~is_mmYYYY]

    dt_mm = pd.to_datetime(mmYYYY, format="%m%Y", errors="coerce")
    dt_yy = pd.to_datetime(yyyyMM, format="%Y%m", errors="coerce")

    out = pd.concat([dt_mm, dt_yy]).sort_index()
    return out + pd.offsets.MonthEnd(0)               # fim do mês

perf["period"] = parse_act_period(perf["ACT_PERIOD"])


### Limpeza de DLQ_STATUS

In [4]:
def clean_dpd(s: pd.Series) -> pd.Series:
    s = (s.astype(str)
           .str.strip()
           .replace({"R": "0", "": np.nan, "XX": np.nan}))
    return pd.to_numeric(s, errors="coerce", downcast="integer")

perf = perf.assign(
    period=parse_act_period(perf["ACT_PERIOD"]),
    dpd=clean_dpd(perf["DLQ_STATUS"]),
).rename(columns={"LOAN_ID": "loan_id"})[["loan_id", "period", "dpd"]]


In [5]:
print(perf.head())
print(perf.dtypes)
assert "period" in perf.columns, "coluna 'period' não foi criada!"

        loan_id     period  dpd
0  100001461640 2007-02-28  0.0
1  100001461640 2007-03-31  0.0
2  100001461640 2007-04-30  0.0
3  100001461640 2007-05-31  0.0
4  100001461640 2007-06-30  0.0
loan_id    string[pyarrow]
period      datetime64[ns]
dpd                float64
dtype: object


In [6]:
# Amostra para testes rápidos
# Exemplo: só 2 % das linhas
perf_sample = perf.sample(frac=0.02, random_state=42)

### Integrando ao TargetBuilder


In [ ]:
from risk_sampler import TargetBuilder

builder = TargetBuilder(
    id_col="loan_id",
    date_col="period",
    dpd_col="dpd",
    # targets desejados, se você tiver um parâmetro opcional:
    targets=["EVER90M12"]
)

targets_df = builder.transform(perf)
display(targets_df.head())
print(targets_df[builder.mapping.keys()].sum())

,loan_id,period,dpd,EVER90M12
69,100001461640,2012-11-30,5.0,0
95,100015135004,2008-07-31,0.0,0
304,100030521552,2016-02-29,0.0,0
395,100036401006,2012-05-31,0.0,0
413,100036401006,2013-11-30,0.0,0


EVER90M12    149
dtype: int64


In [10]:
targets_df['EVER90M12'].value_counts(normalize=True, dropna=False)*100

EVER90M12
0    99.955089
1     0.044911
Name: proportion, dtype: float64

In [13]:
import pandas as pd
import numpy as np

def subsample_by_vintage(
    df: pd.DataFrame,
    target_col: str          = "EVER90M12",
    vintage_col: str         = "period",        # coluna datetime
    min_rate: float          = 0.09,
    max_rate: float          = 0.20,
    random_state: int | None = 42,
    freq: str                = "Q"              # "M" → mês, "Q" → trimestre, "A" → ano
) -> pd.DataFrame:
    """
    Para cada safra (definida por *freq* sobre *vintage_col*):
      • mantem todos os positivos
      • sorteia negativos para que a taxa fique entre [min_rate, max_rate]
        (o alvo dentro do intervalo é aleatório, mudando por safra)
    """
    rng = np.random.default_rng(random_state)
    # cria rótulo de safra: 2007Q1, 2007Q2, …
    vint_lbl = df[vintage_col].dt.to_period(freq)
    df = df.assign(_vintage=vint_lbl)

    out_parts = []
    for vint, g in df.groupby("_vintage", observed=True):
        pos = g[g[target_col] == 1]
        neg = g[g[target_col] == 0]

        # Se não há positivos ou não há negativos, leva tudo
        if pos.empty or neg.empty:
            out_parts.append(g)
            continue

        # escolhe aleatoriamente a taxa alvo desta safra
        target_rate = rng.uniform(min_rate, max_rate)
        n_neg_keep  = int(round(len(pos) * (1 - target_rate) / target_rate))
        n_neg_keep  = min(n_neg_keep, len(neg))

        neg_sample  = neg.sample(n=n_neg_keep, random_state=rng.integers(0, 2**32))

        out_parts.append(pd.concat([pos, neg_sample]))

    out = (
        pd.concat(out_parts)
          .drop(columns="_vintage")
          .sample(frac=1, random_state=random_state)    # embaralha linhas finais
          .reset_index(drop=True)
    )
    return out


In [18]:
frequencia = 'Q'

# quero ~9-20 % de positivos
sub_df = subsample_by_vintage(
    targets_df,
    target_col="EVER90M12",
    vintage_col="period",     # datetime64[ns]
    min_rate=0.09,
    max_rate=0.20,
    random_state=0,
    freq=frequencia                  # por trimestre; troque para "M" se quiser safra mensal
)

# cheque a oscilação por safra
tbl = (
    sub_df.assign(vintage=sub_df["period"].dt.to_period(frequencia))
          .pivot_table(values="EVER90M12", index="vintage", aggfunc="mean") * 100
)
display(tbl.head(10))

,EVER90M12
vintage,
2007Q1,16.666667
2007Q2,9.375000
2007Q3,9.090909
2007Q4,20.000000
2008Q1,15.384615
2008Q2,14.285714
2008Q3,19.047619
2008Q4,9.090909
2009Q1,18.181818


In [2]:
# 1. Caminho para o arquivo
path = "../data/raw/CBSA code_2017.xls"

# 2. Carregue todo o sheet sem header, para descobrir onde está o cabeçalho “verdadeiro”
raw = pd.read_excel(path, sheet_name=0, header=None)

# 3. Encontre a linha onde aparece “CBSA Code”
header_row = raw[
    raw.apply(lambda row: row.astype(str).str.contains("CBSA Code", na=False).any(), axis=1)
].index[0]

# 4. Agora leia de fato, usando essa linha como header e só as colunas A–F
cbsa_df = pd.read_excel(
    path,
    sheet_name=0,
    header=header_row,
    usecols="A:F",
    dtype=str   # lê tudo como string inicialmente
)

# 5. Limpe os nomes de coluna (tirar espaços em branco extras)
cbsa_df.columns = cbsa_df.columns.str.strip()

# 6. Filtre só as linhas cujo “CBSA Code” seja numérico
cbsa_df = cbsa_df[pd.to_numeric(cbsa_df["CBSA Code"], errors="coerce").notnull()]

# 7. Converta para tipos adequados
cbsa_df["CBSA Code"] = cbsa_df["CBSA Code"].astype(int)
# se quiser, deixe área/metadados como string sem NaN:
for col in cbsa_df.columns:
    cbsa_df[col] = cbsa_df[col].fillna("").astype(str).str.strip()

# 8. Renomeie apenas as colunas que existem
rename_map = {
    "CBSA Title": "area_name",
    "Metropolitan/Micropolitan Statistical Area": "area_type",
    "County/County Equivalent": "county",
    "State Name": "state"
}
# preserve só as colunas realmente carregadas e renomeie as que batem
final_cols = []
for orig, new in rename_map.items():
    if orig in cbsa_df.columns:
        final_cols.append((orig, new))

# crie o cbsa_df final com seleção e rename
cbsa_df = cbsa_df[[orig for orig, _ in final_cols]].rename(
    columns={orig: new for orig, new in final_cols}
)

# pronto: cbsa_df agora contém CBSA Code + os campos disponíveis limpos e sem linhas de nota/rodapé
cbsa_df.head()

,area_name,area_type
0,"Aberdeen, SD",Micropolitan Statistical Area
1,"Aberdeen, SD",Micropolitan Statistical Area
2,"Aberdeen, WA",Micropolitan Statistical Area
3,"Abilene, TX",Metropolitan Statistical Area
4,"Abilene, TX",Metropolitan Statistical Area


In [12]:
df

,area_name,area_type
0,"Aberdeen, SD",Micropolitan Statistical Area
1,"Aberdeen, SD",Micropolitan Statistical Area
2,"Aberdeen, WA",Micropolitan Statistical Area
3,"Abilene, TX",Metropolitan Statistical Area
4,"Abilene, TX",Metropolitan Statistical Area
...,...,...
1894,"Yuba City, CA",Metropolitan Statistical Area
1895,"Yuba City, CA",Metropolitan Statistical Area
1896,"Yuma, AZ",Metropolitan Statistical Area
1897,"Zanesville, OH",Micropolitan Statistical Area


In [ ]:
Acquisitions_Variables =["LOAN_ID", "ORIG_CHN", "Seller.Name", "ORIG_RT", "ORIG_AMT", "ORIG_TRM", "ORIG_DTE"
                           ,"FRST_DTE", "OLTV", "OCLTV", "NUM_BO", "DTI", "CSCORE_B", "FTHB_FLG", "PURPOSE", "PROP_TYP"
                           ,"NUM_UNIT", "OCC_STAT", "STATE", "ZIP_3", "MI_PCT", "Product.Type", "CSCORE_C", "MI_TYPE", "RELOCATION_FLG"]

Performance_Variables = ["LOAN_ID", "Monthly.Rpt.Prd", "Servicer.Name", "LAST_RT", "LAST_UPB", "Loan.Age", "Months.To.Legal.Mat"
                          , "Adj.Month.To.Mat", "Maturity.Date", "MSA", "Delq.Status", "MOD_FLAG", "Zero.Bal.Code", 
                          "ZB_DTE", "LPI_DTE", "FCC_DTE","DISP_DT", "FCC_COST", "PP_COST", "AR_COST", "IE_COST", "TAX_COST", "NS_PROCS",
                          "CE_PROCS", "RMW_PROCS", "O_PROCS", "NON_INT_UPB", "PRIN_FORG_UPB_FHFA", "REPCH_FLAG", "PRIN_FORG_UPB_OTH", "TRANSFER_FLG"]

Acquisitions_ColClasses = ["str", "str", "str", "float", "float", "int", "str", "str", "float",
                            "float", "str", "float", "float", "str", "str", "str", "str", "str",
                            "str", "str", "float", "str", "float", "float", "str"]

Performance_ColClasses = ["str", "str", "str", "float", "float", "float", "float", "float", "str",
                            "str", "str", "str", "str", "str", "str", "str", "str",
                            "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "str", "float", "str"]

Acquisitions_dict=dict(zip(Acquisitions_Variables,Acquisitions_ColClasses))
Performance_dict=dict(zip(Performance_Variables,Performance_ColClasses))
#Combined_date_cols=["ORIG_DTE","FRST_DTE","Maturity.Date","ZB_DTE", "LPI_DTE","FCC_DTE"]

Data_A= pd.read_csv('Acquisition_2007Q3.txt', sep='|', 
                    names=Acquisitions_Variables,
                    dtype=Acquisitions_dict,
                    parse_dates=["ORIG_DTE","FRST_DTE"],
                    index_col=False)
Data_P = pd.read_csv('Performance_2007Q3.txt', sep='|', 
                     names=Performance_Variables, 
                     dtype=Performance_dict,
                     parse_dates=["Maturity.Date","ZB_DTE", "LPI_DTE","FCC_DTE"],
                     index_col=False)